In [1]:
import gym
import numpy as np

Vi må først sette opp et miljø. Gym kommer med et sett predefinerte miljøer, hvor 'FrozenLake-v0' er en av dem. Dette er et spill hvor en agent skal navigere en frossen insjø, som er representert med en 4x4 grid.

Eksempel:

SFFF<br>
FHFH<br>
FFFH<br>
HFFG<br>

S: start, trygg<br>
F: frossen overflate, trygg<br>
H: hull, ikke trygg<br>
G: mål, hvor agenten ønsker å komme<br>

En handling består i å bevege seg i en av fire retninger, men ettersom isen er glatt er det ikke garantert at man beveger seg i den retningen man ønsker.

In [2]:
# Definer og sett opp miljøet
env = gym.make('FrozenLake-v0')
env.reset()

# env.action_space og env.obvservation_space definerer tilgjengelige handlinger og observasjoner
print("Action space:", env.action_space)
print("Observation space:", env.observation_space)

# vi kan visualisere et gitt miljø med env.render()
env.render()

Action space: Discrete(4)
Observation space: Discrete(16)

SFFF
FHFH
FFFH
HFFG


<code>env.action_space</code> er altså lik <code>Discrete(4)</code>. Dette vil si at den kan ta fire ulike diskret verdier, som representerer et forsøk på å bevege seg i en av de fire retningene. <code>env.observation_space</code> er lik <code>Discrete(64)</code>, altså 64 ulike verdier som representerer de 8x8=64 mulige posisjonene agenten kan ha.

Det fins mange andre typer handlings- og tilstandsrom. Eksempelvis vil en handling i et handlingsrom lik <code>Box(4,)</code> være en array av fire tall

In [3]:
# Simuler og visualiser fem tilfeldige handlinger
for _ in range(5):
    env.render()
    action = env.action_space.sample()
    observation, reward, done, info = env.step(action)


SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG


Som vi ser returnerer hvert steg fire verdier. De er som følger:

<code>observation</code> En verdi fra observasjonsrommet<br>
<code>reward</code> Et flyttall som representerer fortjenesten for agentens handling, i dette tilfellet 1.0 for målruten og 0.0 for alle andre<br>
<code>done</code> En boolean som forteller oss om spillet er fullført (enten fordi målet er nådd eller agenten har falt ned i et hull)<br>
<code>info</code> En dict med diverse debug-info

In [4]:
print("observation:", observation)
print("reward:", reward)
print("done:", done)
print("info:", info)

observation: 4
reward: 0.0
done: False
info: {'prob': 0.3333333333333333}


Vi kan lære en strategi for dette problemet ved bruk av Q-learning algoritmen.

Epsilon-hyperparametrene under brukes for å kombinere Q-learning med simulated annealing: Vi velger
en tilfeldig handling med en viss sannsynlighet, hvor sannsynligheten for å velge den optimale verdien gitt Q-verditabellen øker med tid

In [25]:
def learn_q(env,
            learning_rate = 0.7,
            gamma = 0.9,
            num_episodes = 20000,
            max_steps = 100,
            min_epsilon = 0.01,
            max_epsilon = 1.0,
            decay_rate = 0.005):
    # Initialiser tabell av Q-verdier og reward for hver episode
    Q = np.zeros([env.observation_space.n, env.action_space.n])
    rewards = []

    epsilon = max_epsilon
    for episode in range(num_episodes):
        state = env.reset()
        reward_total = 0.0
        done = False
        for i in range(max_steps):
            tradeoff = np.random.uniform(0,1)

            # Velg en handling basert på Q-tabell med litt tilfeldighet
            if tradeoff > epsilon:
                action = np.argmax(Q[state,:])
            else:
                action = env.action_space.sample()

            # Utfør handling
            state_new, reward, done, _ = env.step(action)

            # Oppdater Q-verdi for tilstand-handling-par
            Q[state, action] = Q[state, action] + learning_rate*(reward + gamma*np.max(Q[state_new, :]) - Q[state, action])

            # Oppdater total fortjeneste og tilstand
            reward_total += reward
            state = state_new

            # Avslutt dersom spill fullføres
            if done:
                break
        rewards.append(reward_total)
        epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)
    final_reward = np.sum(rewards)/num_episodes
    return Q, final_reward

In [26]:
Q, r = learn_q(env)
print("Endelig resultat:", r)
print("Endelige Q-verdier:", Q)

Endelig resultat: 0.42115
Endelige Q-verdier: [[2.89333840e-02 2.88266634e-02 1.80963059e-02 9.83811005e-02]
 [1.85919214e-02 8.84135650e-04 4.88489247e-03 6.48034045e-02]
 [9.91629694e-02 1.00747591e-03 1.28885714e-03 1.61258151e-03]
 [1.71782126e-04 8.03335852e-04 1.01377067e-03 2.10488690e-03]
 [3.09810587e-02 8.01131422e-03 1.54044117e-02 2.74683598e-02]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [7.33671087e-06 2.89752186e-05 4.02643707e-02 4.10945788e-04]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.54472986e-03 5.50972322e-03 1.70706076e-02 3.37921681e-02]
 [2.96725712e-02 6.35931254e-02 2.68758932e-02 2.57729028e-02]
 [4.91649516e-01 3.10657356e-03 9.46272215e-03 6.63762242e-03]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [2.12733979e-02 2.06103434e-02 1.16673149e-01 7.16116675e-02]
 [7.76448872e-02 3.98974999e-01 2.22043972e-01 2.23671030e-01]
 [0.00000

In [24]:
# Test Q-tabell
s = env.reset()
done = False
while not done:
    env.render()
    a = np.argmax(Q[s,:])
    s, r, done, info = env.step(a)

if(s == 15):
    print("Du klarte det! :D")
    env.render()
else:
    print("Du klarte det ikke :(")
    env.render()


SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Right)
SFFF
FHFH
FFFH
HFFG
  (Down)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Up)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)
SFFF
FHFH
FFFH
HFFG
  (Left)